<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Meta_Learning_for_Rapid_Adaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
python -m venv myenv
source myenv/bin/activate  # On Windows, use `myenv\Scripts\activate`

In [ ]:
pip install torch==1.9.0

In [ ]:
pip install torchmeta

In [ ]:
!pip install torch==1.9.0
!pip install torchmeta

In [ ]:
from torchmeta.datasets.helpers import omniglot
from torchmeta.utils.data import BatchMetaDataLoader
from torchmeta.modules import MetaModule, MetaSequential
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Define a meta-learning compatible network
class MetaModel(MetaModule):
    def __init__(self, input_dim, output_dim):
        super(MetaModel, self).__init__()
        self.net = MetaSequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.net(x)

# Train on few-shot tasks using meta-learning techniques
dataset = omniglot("data", ways=5, shots=1, test_shots=15, meta_train=True, download=True)
meta_loader = BatchMetaDataLoader(dataset, batch_size=16, num_workers=4)

model = MetaModel(input_dim=28*28, output_dim=5)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for batch in meta_loader:
    model.zero_grad()
    train_inputs, train_targets = batch["train"]
    test_inputs, test_targets = batch["test"]

    # Meta-training step
    train_loss = F.cross_entropy(model(train_inputs), train_targets)
    train_loss.backward()
    optimizer.step()

    # Meta-testing step
    with torch.no_grad():
        test_loss = F.cross_entropy(model(test_inputs), test_targets)
        print(f"Meta-test loss: {test_loss.item()}")